In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/used-cars-dataset-from-cardekhocom/Cardekho_Extract.csv
/kaggle/input/used-cars-dataset-from-cardekhocom/cardekho_updated.csv
/kaggle/input/used-cars-dataset-from-cardekhocom/cardekho_imputated.csv


In [2]:
df = pd.read_csv("/kaggle/input/used-cars-dataset-from-cardekhocom/Cardekho_Extract.csv")

In [3]:
df_original = df.copy()

In [4]:
df['seats'] = df['seats'].str[5:]

In [5]:
df[df['seats']=="N/A"]['full_name']

16375    Nissan Kicks Diesel
Name: full_name, dtype: object

In [6]:
df.at[16375, 'seats'] = 5

In [7]:
dropping_df = df[df['seats'].isnull() & df['full_name'].isnull()]
df = pd.concat([df, dropping_df]).drop_duplicates(keep=False)

In [8]:
df['seats'].astype(float)

0        5.0
1        5.0
2        5.0
3        5.0
4        5.0
        ... 
20021    5.0
20022    7.0
20023    5.0
20024    7.0
20025    5.0
Name: seats, Length: 19980, dtype: float64

In [9]:
seat_mode= int(df['seats'].mode())
df['seats'].replace({np.nan: seat_mode}, inplace=True)

In [10]:
df['seats'] = df['seats'].astype(int)

In [11]:
df.dtypes

Source.Name               object
web-scraper-order         object
web-scraper-start-url     object
full_name                 object
selling_price             object
new-price                 object
year                     float64
seller_type               object
km_driven                 object
owner_type                object
fuel_type                 object
transmission_type         object
mileage                   object
engine                    object
max_power                 object
seats                      int64
dtype: object

`df['seats']` is ready 🎉

In [12]:
import re

In [13]:
def engine_power_extractor(text):
    try:
        return re.match(r"(Max Power)([0-9.]+)( bhp)", text).group(2)
    except Exception:
        return np.nan

In [14]:
df['max_power'] = df['max_power'].apply(lambda x: engine_power_extractor(x))

In [15]:
df['max_power'] = df['max_power'].astype(float)
mp_mean = df['max_power'].mean()
df['max_power'].replace(np.nan, mp_mean, inplace=True)

In [16]:
df.describe()

,year,max_power,seats
count,19980.000000,19980.000000,19980.000000
mean,2014.525125,99.447018,5.317868
std,3.249185,43.409940,0.835370
min,1991.000000,25.400000,2.000000
25%,2013.000000,73.970000,5.000000
50%,2015.000000,88.500000,5.000000
75%,2017.000000,112.000000,5.000000
max,2021.000000,626.000000,14.000000


In [17]:
def extractor(text):
    try:
        return re.match(r"(Engine)([0-9]+)( CC)", text).group(2)
    except Exception:
        return np.nan

In [18]:
df['engine'] = df['engine'].apply(lambda x: extractor(x))

In [19]:
df['engine'] = df['engine'].astype(float)

In [20]:
data_mean = df['engine'].mean()

In [21]:
df['engine'].replace(np.nan, data_mean, inplace=True)

In [22]:
var = []
for x in set(df['mileage']):
    var.append(x.split()[-1])
set(var)

{'km/hr', 'km/kg', 'kmpl'}

In [23]:
for x in set(var):
    print(x)
    print(sum(df['mileage'].str.contains(x)))

km/kg
387
km/hr
14
kmpl
19579


In [24]:
df[df['mileage'].str.contains("km/kg")]['fuel_type'].value_counts()

CNG       316
LPG        66
Petrol      5
Name: fuel_type, dtype: int64

In [25]:
df['seller_type'].value_counts()

Dealer              11973
Individual           7817
Trustmark Dealer      190
Name: seller_type, dtype: int64

In [26]:
km_extractor = lambda kms: "".join(list(filter(str.isdigit, kms)))

In [27]:
df['km_driven'] = df['km_driven'].apply(km_extractor)
df['km_driven'] = df['km_driven'].astype(int, )

In [28]:
df['year'] = df['year'].apply(int)
df['year'] = df['year'].astype(int)

In [29]:
df.rename(columns={"new-price": "new_price"}, inplace=True)

In [30]:
import random

In [36]:
def indian_amount_conv(amount):
    if "Lakh" in amount[1]:
        return float(amount[0])  * 100_000
    elif "Cr" in amount[1]:
        return float(amount[0])  * 10_000_000

In [70]:
good_data = []
for row in df['new_price']:
    if type(row) == float:
        good_data.append(row)
        continue
        
    price_data = row[29:]
    if "Cr-" in price_data:
        data_splitted = [price_data.split("-")[0].replace("Cr", "")] + price_data.split("-")[1].split()
        data_splitted[:2] = map(float, data_splitted[:2]) # [1.72, 1.84, 'Cr*']
        amount = [sum(data_splitted[:2]) /2 ] + [data_splitted[-1]] # [15.265, 'Lakh*']; Mean value of the range
        good_data.append(indian_amount_conv(amount)) 

    elif "-" in price_data:
        data_splitted = [price_data.split("-")[0]] + price_data.split("-")[1].split()
        data_splitted[:2] = map(float, data_splitted[:2])
        if data_splitted[0] > data_splitted[1]:
            data_splitted[0] = data_splitted[0]/100
        amount = [sum(data_splitted[:2]) /2 ] + [data_splitted[-1]]
        good_data.append(indian_amount_conv(amount))

    else:
        amount = price_data.split()
        good_data.append(indian_amount_conv(amount))

In [79]:
df.head()

,Source.Name,web-scraper-order,web-scraper-start-url,full_name,selling_price,new_price,year,seller_type,km_driven,owner_type,fuel_type,transmission_type,mileage,engine,max_power,seats,new_price_formatted
0,cardekho_extract(0-2000).csv,1611917819-1662,https://www.cardekho.com/used-car-details/used...,Maruti Alto Std,1.2 Lakh*,NaN,2012,Individual,120000,First Owner,Petrol,Manual,Mileage19.7 kmpl,796.0,46.30,5,NaN
1,cardekho_extract(0-2000).csv,1611918361-1902,https://www.cardekho.com/used-car-details/used...,Hyundai Grand i10 Asta,5.5 Lakh*,New Car (On-Road Price) : Rs.7.11-7.48 Lakh*,2016,Individual,20000,First Owner,Petrol,Manual,Mileage18.9 kmpl,1197.0,82.00,5,729500.0
2,cardekho_extract(0-2000).csv,1611917012-1306,https://www.cardekho.com/used-car-details/used...,Hyundai i20 Asta,2.15 Lakh*,NaN,2010,Individual,60000,First Owner,Petrol,Manual,Mileage17.0 kmpl,1197.0,80.00,5,NaN
3,cardekho_extract(0-2000).csv,1611917695-1607,https://www.cardekho.com/used-car-details/used...,Maruti Alto K10 2010-2014 VXI,2.26 Lakh*,NaN,2012,Individual,37000,First Owner,Petrol,Manual,Mileage20.92 kmpl,998.0,67.10,5,NaN
4,cardekho_extract(0-2000).csv,1611914861-367,https://www.cardekho.com/used-car-details/used...,Ford Ecosport 2015-2021 1.5 TDCi Titanium BSIV,5.7 Lakh*,New Car (On-Road Price) : Rs.10.14-13.79 Lakh*,2015,Dealer,30000,First Owner,Diesel,Manual,Mileage22.77 kmpl,1498.0,98.59,5,1196500.0


In [88]:
for price in set(df['selling_price']):
    if not ("Lakh" in price or "Cr" in price):
        print(price)

81,000*
51,000*
99,111*
70,000*
50,000*
77,000*
78,000*
93,000*
34,000*
57,000*
95,585*
89,000*
60,000*
66,000*
30,000*
95,000*
59,000*
25,000*
75,000*
96,000*
71,000*
82,875*
32,500*
41,694*
88,000*
56,000*
85,000*
40,000*
92,000*
74,999*
94,000*
35,000*
58,000*
82,000*
55,000*
68,000*
69,000*
97,000*
45,000*
93,750*
46,288*
86,250*
42,000*
82,608*
65,000*
62,000*
91,000*
87,000*
72,000*
80,000*
90,000*
98,000*
72,308*
52,000*
53,000*
45,946*
74,000*
99,000*


In [103]:
price_data = []
for price in df['selling_price']:
    if ("Lakh" in price or "Cr" in price):
        price = price.split()
        price_data.append(indian_amount_conv(price))
    else:
        float(price[:-1].replace(",", ""))
        price_data.append(price)

In [104]:
df["selling_price_formatted"] = price_data

In [110]:
df.to_csv("Formatted Indian Used Car.csv")